## AWS のサービスを使ったサーバレスな前処理、学習、推論

このノートブックでは、AWS のマネージドな機械学習基盤サービスである、Amazon SageMaker を活用して、前処理や学習、推論を行います。SageMaker を活用すると、基本的にはデータは Amazon S3 に保存、実行環境はコンテナ化して Amazon ECR へ登録、処理の内容はほぼ従来どおり、Python などスクリプトで記述して、必要なタイミングでジョブを実行し、処理が終われば、インスタンスが停止してジョブが終わるというサーバーレスに近いジョブ実行環境を構築できます。それぞれのデータサイズが大きい、学習に時間がかかる、など Jupyter 環境では処理がしにくいような計算を行いたい場合には有用です。

In [ ]:
# !pip install -U --quiet "sagemaker>=2.19.0"

### 使用するライブラリなどの読み込み

In [ ]:
import pandas as pd

import sagemaker
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

# SageMaker を活用するための権限が付与された Role を準備します。
role = get_execution_role()

### Amazon S3 へのデータのアップロード

データを S3 にアップロードし、AWS のサービスから活用する準備をします。

In [ ]:
sagemaker_session = sagemaker.Session()
input_train = sagemaker_session.upload_data(path='./data/train.csv', key_prefix='kaggle-ml-pipeline/data')
input_test = sagemaker_session.upload_data(path='./data/test.csv', key_prefix='kaggle-ml-pipeline/data')

## データの前処理
### データ前処理用のジョブのためのコンテナの作成

データの前処理や特徴量作成をジョブとして実行するために SageMaker Proccessing というサービス(詳細は後述)を使います。 SageMaker では組み込みコンテナとして [Apache Spark](https://docs.aws.amazon.com/sagemaker/latest/dg/use-spark-processing-container.html) と [scikit-learn](https://docs.aws.amazon.com/sagemaker/latest/dg/use-scikit-learn-processing-container.html) などが準備されていますが、必要なライブラリを自分たちで持ち込んだコンテナを準備したくなる場合もあると思います。そういった場合には、独自のコンテナを活用することも可能なので、イメージを[作成](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-container-run-scripts.html)して活用します。手順は下記です。
- 1. Jupyter ノートブック上でイメージをビルドする
- 2. Amazon Elastic Container Registry の リポジトリにイメージを push する
- 3. push したイメージを利用して前処理を行う

活用するイメージは Dockerfile で定義し、`docker` コマンドでビルドします。

In [ ]:
!cat ./scripts/preprocess/Dockerfile

In [ ]:
!docker build -t sagemaker-kaggle-titanic-preprocess ./scripts/preprocess

コンテナをコンテナレジストリである、ECR へアップロードします。この際、AWS の python SDK である boto3 を使います。

In [ ]:
import boto3

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
tag = ':latest'

# Repository 名の中に sagemaker が含まれている必要がある
ecr_repository = f'sagemaker-kaggle-titanic-preprocess'
image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repository+tag}'

!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr create-repository --repository-name $ecr_repository
 
!docker build -t {ecr_repository} .
!docker tag {ecr_repository + tag} $image_uri
!docker push $image_uri

print(f'コンテナは {image_uri} へ登録されています。')

これで前処理に必要な Docker イメージが準備できました。

### 前処理を実行する SageMaker Processing ジョブを定義して実行
Amazon SageMaker Processingは機械学習の前処理、後処理、モデル評価とそれぞれの計算処理をAmazon SageMaker の機能です。実行環境（コンテナ）を用意し、実行したい処理を記述し、実行するインスタンスタイプを指定することでジョブを実行することができます。コンテナと処理を記述したスクリプトを実行し、処理が終わるとインスタンスが停止するというシンプルな機能から、上述した機械学習関連のデータ処理にとどまらず、学習や推論にもお使い頂くような場合もあります。ジョブの定義や実行を行う Python SDK の詳細については[ドキュメント](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_processing.html)を確認して下さい。

In [ ]:
job_name = f'sagemaker-kaggle-preprocessing-train'
processing_input_dir = '/opt/ml/processing/input'
processing_output_dir = '/opt/ml/processing/output'
output_s3_path = 's3://' + sagemaker_session.default_bucket() + '/kaggle-ml-pipeline'

output_s3_path_preprocess = output_s3_path + '/preprocessed'

前処理を行うための Processor インスタンスを作成します。

In [ ]:
processor = ScriptProcessor(
                    base_job_name=job_name,
                    image_uri=image_uri,
                    command=['python3'],
                    role=role,
                    instance_count=1,
                    instance_type='ml.c5.xlarge'
                          )

Processing ジョブの実行をします。入力データに関しては、入力元の S3 のパスと、コンテナ上のどこに展開するかを定義、出力データには、コンテナ上のどのデータを S3 のどこに保存するかを定義。処理する内容は、[1_train_inference_notebook.ipynb](https://github.com/tkazusa/kaggle-mlpipeline-titanic/blob/main/notebooks/1_train_inference_notebook.ipynb) と同様のものを、スクリプト化し、`./scripts/preprocess/preprocess_script/` へ置いてあり、それを活用しています。スクリプトへ渡すコマンドライン引数は、 `run` メソッドの `arguments` 引数で定義しています。

In [ ]:
processor.run(
    code='./scripts/preprocess/preprocess_script/preprocess.py', # S3 の URI でも可
    inputs=[ProcessingInput(source=input_train, destination=processing_input_dir)],
    outputs=[ProcessingOutput(source=processing_output_dir, destination=output_s3_path_preprocess)],
    arguments=[
          '--data_type', 'train',
          '--input_dir',processing_input_dir,
          '--output_dir',processing_output_dir
              ]
            )

この前処理ジョブによって `s3:<bucket-name>/kaggle-ml-pipeline/data/train.csv` にあった元のデータが、特徴量が追加されて、 `s3:<bucket-name>/kaggle-ml-pipeline/preprocessed/train.csv` へ保存されました。

## 学習

機械学習モデルの学習には、SageMaker の学習ジョブを活用します。この学習ジョブは先程の前処理を行った Processing ジョブと異なり分散学習や実験管理など、学習に必要な機能を簡単に追加することもできます。そのため、学習用のジョブとして独立した機能が提供されています。今回は SageMaker が提供している、`scikit-learn` が事前にインストールされたコンテナを実行環境として使います。 Processing ジョブと同様に独自コンテナを持ち込むこともできます。詳細は[コチラ](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/docker-containers.html)をご確認下さい。

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

output_s3_path_train = output_s3_path + '/train'

sklearn = SKLearn(
    entry_point='scripts/train/train.py',
    framework_version="0.23-1",
    instance_type="ml.m5.xlarge",
    output_path=output_s3_path_train,
    role=role)

In [ ]:
train_input = output_s3_path_preprocess + '/train.csv'
sklearn.fit({'train': train_input})

### 推論

テスト用データにも前処理を施します。先程と同様のスクリプトとコンテナを活用します。

In [ ]:
job_name = f'sagemaker-kaggle-preprocessing-test'

processor = ScriptProcessor(base_job_name=job_name,
                                   image_uri=image_uri,
                                   command=['python3'],
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.c5.xlarge'
                                  )



processor.run(code='./scripts/preprocess/preprocess_script/preprocess.py', # S3 の URI でも可
              inputs=[ProcessingInput(source=input_test, destination=processing_input_dir)],
              outputs=[ProcessingOutput(source=processing_output_dir, destination=output_s3_path_preprocess)],
              arguments=[
                  '--data_type', 'test',
                  '--input_dir',processing_input_dir,
                  '--output_dir',processing_output_dir
                      ]
                    )

推論には SageMaker のバッチ変換ジョブを使います。

In [ ]:
output_s3_path_inference = output_s3_path + '/batch_inference'

transformer = sklearn.transformer(
                        instance_count=1,
                        instance_type='ml.m5.xlarge',
                        output_path=output_s3_path_inference
)

In [ ]:
test_input = output_s3_path_preprocess + '/test.csv'

transformer.transform(
    data=test_input,
    content_type='text/csv')

print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)

transformer.wait()